In [ ]:
!wget /resources/data/latimeshealth.txt https://www.dropbox.com/s/sigw88k0a4f0o9g/latimeshealth.txt?dl=0

/resources/data/latimeshealth.txt: Scheme missing.
--2021-11-16 00:43:56--  https://www.dropbox.com/s/sigw88k0a4f0o9g/latimeshealth.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/sigw88k0a4f0o9g/latimeshealth.txt [following]
--2021-11-16 00:43:56--  https://www.dropbox.com/s/raw/sigw88k0a4f0o9g/latimeshealth.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc02b716cc5d71c639b691ace664.dl.dropboxusercontent.com/cd/0/inline/BaCZtkjrqE6ZanvOp8OI_aslWFBGF1GUsDhRgYhGSDePQuoAnYcEXIttOQZueJ_xaQeJh757rLMtMJ6ewMpnectRRfmyCf8WzknKrIov2bYepj3JRWkt9x6nDqqrDCBva1wIWRyDFIrfITCYP3umXgZk/file# [following]
--2021-11-16 00:43:56--  https://uc02b716cc5d71c639b691ace664.dl.dropboxusercontent.com/cd/0/inline/BaCZtkjrqE6ZanvOp8O

In [ ]:
import pandas as pd

data = pd.read_csv('/content/latimeshealth.txt?dl=0', sep="|", header=None)
data.columns = ["tid", "timestamp", "tweet"]

In [ ]:
# dropping timestamp and tweet id
data = data.drop('tid', 1)
data = data.drop('timestamp', 1)

In [ ]:
import re
pd.set_option('display.max_colwidth',1000)

# preprocessing
def preprocess(tweet):

    tweet = re.sub('http\S+|www.\S+', '', tweet)
    tweet = re.sub('@[A-Za-z0-9]+', '', tweet)
    tweet = re.sub('#', '', tweet)
    tweet = tweet.lower()

    return tweet

data['tweet'] = data['tweet'].apply(preprocess)

tweet_list = data.values.tolist()
print(tweet_list)

[['five new running shoes that aim to go the extra mile '], ['gym rat: disq class at crunch is intense workout on pulley system '], ['noshing through thousands of ideas at natural products expo west '], ['natural products expo also explores beauty, supplements and more '], ['free fitness weekends in south bay beach cities aim to spark activity '], ['for kayla harrison, sacrifices paid off with judo gold '], ["sonic waves a treatment for alzheimer's disease? "], ["an ultrasound for the brain restores memory in alzheimer's w/o so much as a needle. but only--so far--in mice."], ["is apple's new researchkit really a boon for medical research? "], ['warning to chantix users: if you drink while taking this drug, you might not remember it the next day. '], ['some chantix users had memory problems after drinking a modest amount of alcohol, _fda says '], ["guanine isn't just an ingredient in dna. in nanocrystal form, it allows chameleons to change colors.  "], ['the _fda has documented 64 cases

In [ ]:
import math
import random
def jaccardDistance(t1, t2):
    
    tweet1 = t1[0]
    tweet2 = t2[0]

    tweet1 = set(tweet1.split(' '))
    tweet2 = set(tweet2.split(' '))
    inter = tweet1.intersection(tweet2)
    uni = tweet1.union(tweet2)

    # return the jaccard distance
    return (1 - (len(inter) / len(uni)))

def converge(old_centers, new_centers):

  if len(old_centers) != len(new_centers):
        return False

  # see if the centers are the same or not
  for c in range(len(new_centers)):
      if " ".join(new_centers[c]) != " ".join(old_centers[c]):
          return False
    # return true as long as centers are the same
  return True

def cluster(tweets, centers):

# dictionary to store clusters, their tweets, and the minimum distance from the centers
    clusters = dict()

    # loop through every tweet and cluster it
    for tid in range(len(tweets)):
        
        # keeps track of minimum distance from a cluster and the current cluster it is in
        minimum_distance = math.inf
        cluster_id = -1;

        # find closest center to cluster        
        for cen in range(len(centers)):
            current_distance = jaccardDistance(centers[cen], tweets[tid])

            # distance is 0 if tweets are the same
            if centers[cen] == tweets[tid]:
                cluster_id = cen
                minimum_distance = 0
                break

            # update the minimum distance and cluster id if you find a center closer
            if current_distance < minimum_distance:
                cluster_id = cen
                minimum_distance = current_distance

        # if jaccard distance is 1 for all centers, randomly assign to a cluster
        if minimum_distance == 1:
            cluster_id = random.randint(0, len(centers) - 1)

        # assign the tweet to a cluster
        clusters.setdefault(cluster_id, []).append([tweets[tid]])
        # getting the id of last tweet in the cluster to save the distance
        last_tweet_id = len(clusters.setdefault(cluster_id, [])) - 1
        # adding minimum distance to the dictionary
        clusters.setdefault(cluster_id, [])[last_tweet_id].append(minimum_distance)

    return clusters

def centers_calculate(clusters):

    centers = []

  # loop through all the tweets in the cluster and find the one with the least distance from all the other tweets in the same cluster
    for c in range(len(clusters)):

      # holds the minimum distance from every other tweet
        total_minimum_distance = math.inf
        # holds the id of the current center
        center_id = -1

        # holds the minimum distances
        minimum_distances = []

        for tid1 in range(len(clusters[c])):
            minimum_distances.append([])
            # holds the sum of distance from every other tweet
            distance = 0
            # get distance from every tweet to every other tweet
            for tid2 in range(len(clusters[c])):
                if tid1 != tid2:
                  # we have already calculated this so no need to do it again
                    if tid2 < tid1:
                        tempdistance = minimum_distances[tid2][tid1]
                    else:
                        tempdistance = jaccardDistance(clusters[c][tid1][0], clusters[c][tid2][0]) # calculate jaccard distance

                    # save the minimum distance
                    minimum_distances[tid1].append(tempdistance)
                    # adding the to sum of distances
                    distance += tempdistance
                else:
                    minimum_distances[tid1].append(0) # if the tweets are the same, the distance is 0

            #  update the center if you find a tweet that is closer to all the other tweets
            if distance < total_minimum_distance:
                total_minimum_distance = distance
                center_id = tid1

        # add the tweet to the list of centers
        centers.append(clusters[c][center_id][0])

    return centers

def SSE(clusters):

    sse = 0
    # we have already saved the distances in the clusters dictionary, so loop through and compute sse
    for c in range(len(clusters)):
        for tid in range(len(clusters[c])):
            sse = sse + (clusters[c][tid][1] * clusters[c][tid][1])

    return sse


In [ ]:
def kmeans(k):
  # initializing to random centers
  centers = random.sample(range(len(tweet_list)), k)

  center_tweets = []

  for i in centers:
    center_tweets.append(tweet_list[i])

  old_center_tweets = []

  # keep track of iterations, in case it takes a long time to converge
  current_iter = 0

  # keep looping while it hasn't converged or there aren't more than 50 iterations(in case it converges very slowly)
  while(converge(old_center_tweets, center_tweets) == False and current_iter < 50):
    clusters = cluster(tweet_list, center_tweets)
    old_center_tweets = center_tweets
    center_tweets = centers_calculate(clusters)
    current_iter = current_iter + 1

  if(current_iter == 50):
      print("50 iterations reached and not yet converged.")
  else:
      print("Finished converging")

  sum_squared_error = SSE(clusters)

  for i in range(0,k):
    print(len(clusters[i]))
  print(sum_squared_error)

In [ ]:
kmeans(5)
kmeans(10)
kmeans(15)
kmeans(20)
kmeans(25)

Finished converging
680
989
592
1178
732
3160.3518489701155
Finished converging
544
481
738
312
281
138
338
30
515
794
3095.1072553775025
Finished converging
135
131
162
198
256
485
406
557
207
27
172
458
426
168
383
3050.814152626312
Finished converging
547
174
129
68
176
503
259
46
352
121
44
565
225
103
61
241
189
117
173
78
3002.5249291815394
Finished converging
487
239
114
403
142
224
106
113
79
79
136
96
132
93
45
97
71
247
59
24
141
324
62
355
303
2978.928080954142


# New Section